In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
import pandas as pd


np.random.seed(298)

Sort the iterations data based on the `id` column so that we can sample by index. We will also grab the `holdout` column so that we have that as a separate object (removing it from the `probs` object).

In [16]:
probs['holdout']

0       1
1       1
2       1
3       1
4       1
       ..
2876    0
2877    0
2878    0
2879    0
2880    0
Name: holdout, Length: 2881, dtype: int64

In [18]:
probs = pd.read_csv('data/iterations.csv')
holdout = probs[['id', 'holdout']]


In [19]:
probs = probs.drop('holdout', axis=1)

In [21]:
probs = probs.sort_values(by="id", ascending=True)

In [24]:
probs.shape[1]

10001

In [29]:
probs_ = probs[[col for col in probs.columns if 'iter' in col]].to_numpy()

In [41]:
list(probs_[:, 0]) == probs.iter1.tolist()

True

[1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,


In [1]:
def remove_cols(arr):
    """
    Eliminates any column in the array where all elements are zero.

    Parameters:
    - arr: numpy array of any shape with 2 dimensions

    Returns:
    - numpy array with columns where all elements are zero removed
    """
    # Check for columns where all elements are zero
    non_zero_columns = ~np.all(arr == 0, axis=0)
    
    return arr[:, non_zero_columns]

# inputs and outputs
X = remove_cols(np.genfromtxt('data/orth.csv', delimiter=","))
Y = remove_cols(np.genfromtxt('data/phon.csv', delimiter=","))

2024-02-04 21:17:35.090915: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-04 21:17:35.124655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
probs.columns.tolist()[0:10]

['id',
 'iter1',
 'iter2',
 'iter3',
 'iter4',
 'iter5',
 'iter6',
 'iter7',
 'iter8',
 'iter9']

In [13]:
probs.id.min()

1

In [6]:

optimizer = Adam(learning_rate=.1)
hidden = 100

model = Sequential()
model.add(Dense(hidden, input_shape=(X.shape[1],)))
model.add(Dense(Y.shape[1], activation='sigmoid'))
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy', 'mse'])

In [7]:

import time

start_time = time.time()
# Calculate the difference
model.fit(X, Y, epochs=300)
end_time = time.time()
compute_time = end_time - start_time

print(f"The operation took {compute_time} seconds.")


Epoch 1/300
91/91 [==============================] - 0s 2ms/step - loss: 0.1854 - accuracy: 0.0396 - binary_crossentropy: 0.1854 - mse: 0.0468
Epoch 2/300
91/91 [==============================] - 0s 2ms/step - loss: 0.1494 - accuracy: 0.1972 - binary_crossentropy: 0.1494 - mse: 0.0320
Epoch 3/300
91/91 [==============================] - 0s 2ms/step - loss: 0.1871 - accuracy: 0.4408 - binary_crossentropy: 0.1871 - mse: 0.0342
Epoch 4/300
91/91 [==============================] - 0s 2ms/step - loss: 0.2106 - accuracy: 0.6380 - binary_crossentropy: 0.2106 - mse: 0.0326
Epoch 5/300
91/91 [==============================] - 0s 2ms/step - loss: 0.2555 - accuracy: 0.7209 - binary_crossentropy: 0.2555 - mse: 0.0345
Epoch 6/300
91/91 [==============================] - 0s 2ms/step - loss: 0.2288 - accuracy: 0.7661 - binary_crossentropy: 0.2288 - mse: 0.0310
Epoch 7/300
91/91 [==============================] - 0s 2ms/step - loss: 0.3191 - accuracy: 0.7893 - binary_crossentropy: 0.3191 - mse: 0.0332

In [ ]:
model.evaluate(X, Y)

In [8]:
60*60*24


86400